<a href="https://colab.research.google.com/github/MohitSolunke/Google_Colab_Activities/blob/main/Text_Summerizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset
news = load_dataset("multi_news", split = 'test')

In [3]:
news.to_pandas()

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...
...,...,...
5617,Tweet with a location \n \n You can add locati...,– The traditional end-of-summit group photo at...
5618,Loic Venance/AFP/Getty Images \n \n The awards...,– Sofia Coppola scored a historic victory at t...
5619,(CNN) A federal criminal investigation into a ...,– The duck boat sinking that killed 17 on a Mi...
5620,An archive of the public statements deleted by...,– Note to tweeting politicians: Watch what you...


In [4]:
train_news, test_news = news.train_test_split(test_size=0.2).values()

In [5]:
train_news.to_pandas()

,document,summary
0,Rotisserie chickens have been around for a whi...,– Those who like chicken might have noticed a ...
1,NEW YORK (CBSNewYork) — A mother has been char...,"– After a months-long investigation, an Alabam..."
2,"It is often suggested that Facebook, Google, a...",– Imagine you're in the middle of law class le...
3,Go to the Legal Help page to request content c...,– A car plowed into a crowd of children outsid...
4,Modern slot machines develop an unbreakable ho...,– About 1% of the US population (around 3.2 mi...
...,...,...
4492,These crawls are part of an effort to archive ...,– Alleged Waffle House mass shooter Travis Rei...
4493,North Carolina State Archives and State Librar...,"– Writer Salman Rushdie was furious at ""moroni..."
4494,John Thune isn't running for president. \n \n ...,– The 2012 crop of GOP presidential candidates...
4495,"If you’re serious about crafting, then you may...","– When customers order an heirloom pet teepee,..."


In [6]:
!pip install transformers accelerate

In [7]:
from transformers import AutoTokenizer


In [8]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [9]:
def preprocessing(examples):
  input = ['summary' + doc for doc in examples['document']]
  model_input = tokenizer(input, max_length = 1024, truncation = True)
  label = tokenizer( text = examples['summary'], max_length = 128, truncation = True)
  model_input['labels'] = label['input_ids']
  return model_input

In [10]:
tokenized = train_news.map(preprocessing, batched = True)

Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = 't5-small')

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "./results",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 10,
    per_device_eval_batch_size = 10,
    weight_decay = 0.01,
    num_train_epochs = 10,
    fp16 = True,


)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized,
    tokenizer = tokenizer,
    data_collator = data_collator,
)

In [ ]:
trainer.train()